## Data Preparation

In [1]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.transforms.functional as F
from tqdm import tqdm
from PIL import Image
import shutil
import random
import os
import glob

def style_sampling(base_path, dest_path, n_samples=100, split=(0.8, 0.1, 0.1)):
    """
    Lấy mẫu ảnh từ mỗi style và chia đều train/valid/test.
    Đặt tên file dạng style_001.jpg, style_002.jpg...
    
    Args:
        base_path (str): thư mục chứa các folder style
        dest_path (str): thư mục lưu kết quả
        n_samples (int): số ảnh lấy mẫu cho mỗi style
        split (tuple): tỉ lệ chia train/valid/test
    """
    random.seed(2025)
    
    subsets = ["train", "valid", "test"]
    for subset in subsets:
        os.makedirs(os.path.join(dest_path, subset), exist_ok=True)
    
    styles = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    total_copied = 0

    for style in tqdm(styles, desc="Sampling styles"):
        style_path = os.path.join(base_path, style)
        # Lọc file ảnh
        image_files = [f for f in os.listdir(style_path) 
                       if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
        if not image_files:
            continue

        # Lấy mẫu ngẫu nhiên
        sample_imgs = random.sample(image_files, min(len(image_files), n_samples))
        n = len(sample_imgs)

        # Số lượng cho train/valid/test (làm tròn xuống)
        n_train = int(n * split[0])
        n_valid = int(n * split[1])
        n_test = n - n_train - n_valid

        # Chia ảnh
        split_dict = {
            "train": sample_imgs[:n_train],
            "valid": sample_imgs[n_train:n_train+n_valid],
            "test": sample_imgs[n_train+n_valid:]
        }

        # Copy ảnh sang folder tương ứng với tên style_index.jpg
        for subset, imgs in split_dict.items():
            dest_folder = os.path.join(dest_path, subset)
            for idx, img_name in enumerate(imgs, start=1):
                src = os.path.join(style_path, img_name)
                dst_name = f"{style}_{idx:03d}.jpg"  # style_001.jpg, style_002.jpg...
                dst = os.path.join(dest_folder, dst_name)
                shutil.copy(src, dst)
                total_copied += 1

    print(f"Đã sao chép {total_copied} ảnh từ {len(styles)} style vào {dest_path}/train, valid, test")


# ------------------- Transform -------------------
class TransformImageNet:
    def __init__(self, target_long=512, min_short=256, crop_size=None, gray_ratio=0.0):
        """
        target_long: cạnh lớn của ảnh sau resize
        min_short: nếu cạnh nhỏ < min_short, sẽ padding
        crop_size: nếu muốn crop, None = không crop
        gray_ratio: xác suất chuyển ảnh sang grayscale
        """
        self.target_long = target_long
        self.min_short = min_short
        self.crop_size = crop_size
        self.gray_ratio = gray_ratio
        self.to_tensor = T.ToTensor()
        self.normalize = T.Normalize(mean=[0.485,0.456,0.406],
                                     std=[0.229,0.224,0.225])

    def resize_and_pad(self, img):
        w, h = img.size
        # scale tỉ lệ theo cạnh lớn
        if w > h:
            new_w = self.target_long
            new_h = int(h * self.target_long / w)
        else:
            new_h = self.target_long
            new_w = int(w * self.target_long / h)
        img = img.resize((new_w, new_h), Image.Resampling.LANCZOS)

        # padding nếu cạnh nhỏ < min_short
        pad_w = max(0, self.min_short - new_w)
        pad_h = max(0, self.min_short - new_h)
        if pad_w > 0 or pad_h > 0:
            img = F.pad(img, (0,0,pad_w,pad_h), fill=0)
        return img

    def __call__(self, img):
        # Grayscale augmentation
        if random.random() < self.gray_ratio:
            img = img.convert("L").convert("RGB")

        img = self.resize_and_pad(img)

        # RandomCrop nếu muốn
        if self.crop_size:
            img = T.RandomCrop(self.crop_size)(img)

        img = self.to_tensor(img)
        img = self.normalize(img)
        return img

# ------------------- Dataset -------------------
class CustomImageDataset(Dataset):
    def __init__(self, content_folder, style_folder, subset,
                 transform=None, gray_ratio=0.2,
                 valid_ext=('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
        self.content_folder = os.path.join(content_folder, subset)
        self.style_folder = os.path.join(style_folder, subset)

        self.content_files = []
        self.style_files = []

        for ext in valid_ext:
            self.content_files.extend(glob.glob(os.path.join(self.content_folder, f"*{ext}")))
            self.style_files.extend(glob.glob(os.path.join(self.style_folder, f"*{ext}")))

        self.content_files = sorted(self.content_files)
        self.style_files = sorted(self.style_files)

        if len(self.content_files) == 0:
            raise RuntimeError(f"No content images found in {self.content_folder}")
        if len(self.style_files) == 0:
            raise RuntimeError(f"No style images found in {self.style_folder}")

        self.transform = transform
        self.gray_ratio = gray_ratio

    def __len__(self):
        return len(self.content_files)

    def __getitem__(self, idx):
        # Content image
        content_path = self.content_files[idx]
        content_img = Image.open(content_path).convert("RGB")
        
        # Style image (random)
        style_path = random.choice(self.style_files)
        style_img = Image.open(style_path).convert("RGB")
        
        if self.transform:
            content_img = self.transform(content_img)
            style_img = self.transform(style_img)

        return content_img, style_img

# ------------------- DataLoader factory -------------------
def get_dataloaders(content_folder, style_folder,
                    batch_size=8, num_workers=4, 
                    persistent_workers=False, gray_ratio=0.2,
                    target_long=512, min_short=256, crop_size=256):
    """
    Tạo DataLoader cho train/valid/test.
    Giả sử content_folder và style_folder đã có subfolder 'train', 'valid', 'test'.
    Có thể bật tqdm để quan sát tiến trình load dữ liệu.
    """
    transform = TransformImageNet(
        target_long=target_long,
        min_short=min_short,
        crop_size=crop_size,
        gray_ratio=gray_ratio
    )

    loaders = {}
    for subset in ["train", "valid", "test"]:
        dataset = CustomImageDataset(
            content_folder,
            style_folder,
            subset=subset,
            transform=transform,
            gray_ratio=gray_ratio,
        )
        shuffle = (subset == "train")

        loader = DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=num_workers,
            pin_memory=True,
            persistent_workers=persistent_workers
        )

        loaders[subset] = loader

    return loaders

## AdaIn

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models

class VGGEncoder(nn.Module):
    def __init__(self, path_vgg_weights=None, device='cpu'):
        super(VGGEncoder, self).__init__()
        self.device = device  # lưu device để dùng khi forward
        if path_vgg_weights is None:
            vgg16 = models.vgg16(pretrained=True)
        else:
            vgg16 = models.vgg16()
            vgg16.load_state_dict(torch.load(path_vgg_weights, map_location=device))

        # Chỉ lấy feature tới conv4_3
        self.encoder_layers = nn.Sequential(*list(vgg16.features.children())[:21])

        # Freeze weights
        for param in self.encoder_layers.parameters():
            param.requires_grad = False

        # Chuyển encoder sang device
        self.encoder_layers.to(device)  

    def forward(self, x):
        x = x.to(self.device)
        x = self.encoder_layers(x)
        return x

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(channels, channels, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(channels, channels, kernel_size=3)
        )

    def forward(self, x):
        return x + self.block(x)

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_channels, out_channels, 3),
            nn.ReLU(inplace=True),
            ResidualBlock(out_channels)
        )

    def forward(self, x):
        return self.block(x)
    
class Decoder(nn.Module):
    def __init__(self, out_channels=3):
        super().__init__()
        self.decoder_layers = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(512, 256, 3),
            nn.ReLU(inplace=True),

            ConvBlock(256, 256),
            nn.ReflectionPad2d(1),
            nn.Conv2d(256, 256, 3),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(256, 128, 3),
            nn.ReLU(inplace=True),

            ConvBlock(128, 128),
            nn.ReflectionPad2d(1),
            nn.Conv2d(128, 64, 3),
            nn.ReLU(inplace=True),

            ConvBlock(64, out_channels)
        )

    def forward(self, x):
        return self.decoder_layers(x)

    
class AdaINet(nn.Module):
    def __init__(self, path_vgg_weights=None, out_channels=3, device='cpu'):
        super(AdaINet, self).__init__()
        self.encoder = VGGEncoder(path_vgg_weights=path_vgg_weights, device=device)
        self.decoder = Decoder(out_channels=out_channels)

    def forward(self, content, style, alpha=1.0):
        # Encode
        content_feat = self.encoder(content)
        style_feat = self.encoder(style)
        # AdaIN
        t = self.adain(content_feat, style_feat)
        t = alpha * t + (1 - alpha) * content_feat
        # Decode
        generated = self.decoder(t)
        return generated, t

    def adain(self, content_feat, style_feat, eps=1e-5):
        c_mean = torch.mean(content_feat, dim=[2, 3], keepdim=True)
        c_std = torch.std(content_feat, dim=[2, 3], keepdim=True) + eps
        s_mean = torch.mean(style_feat, dim=[2, 3], keepdim=True)
        s_std = torch.std(style_feat, dim=[2, 3], keepdim=True) + eps

        normalized_feat = (content_feat - c_mean) / c_std
        stylized_feat = normalized_feat * s_std + s_mean
        return stylized_feat

    
class VGGEncoderMultiLayer(nn.Module):
    def __init__(self, path_vgg_weights=None, device='cpu'):
        super(VGGEncoderMultiLayer, self).__init__()
        self.device = device
        if path_vgg_weights is None:
            vgg16 = models.vgg16(pretrained=True)
        else:
            vgg16 = models.vgg16()
            vgg16.load_state_dict(torch.load(path_vgg_weights, map_location=device))
        
        self.slice1 = nn.Sequential(*list(vgg16.features.children())[:2])   # relu1_1
        self.slice2 = nn.Sequential(*list(vgg16.features.children())[2:7])  # relu2_1
        self.slice3 = nn.Sequential(*list(vgg16.features.children())[7:14]) # relu3_1
        self.slice4 = nn.Sequential(*list(vgg16.features.children())[14:21]) # relu4_1
        # Freeze weights
        for param in self.parameters():
            param.requires_grad = False

        self.to(device)
        
    def forward(self, x):
        x = x.to(self.device)
        relu1_1 = self.slice1(x)
        relu2_1 = self.slice2(relu1_1)
        relu3_1 = self.slice3(relu2_1)
        relu4_1 = self.slice4(relu3_1)
        return {
            'relu1_1': relu1_1,
            'relu2_1': relu2_1,
            'relu3_1': relu3_1,
            'relu4_1': relu4_1
        }
class AdaINLossMultiLayer(nn.Module):
    def __init__(self, encoder, alpha=1.0, beta=0.5, eps=1e-5):
        super().__init__()
        self.encoder = encoder
        self.alpha = alpha
        self.beta = beta
        self.eps = eps
        self.style_layers = ['relu1_1', 'relu2_1', 'relu3_1', 'relu4_1']

    def calc_mean_std(self, feat):
        B, C = feat.size(0), feat.size(1)
        feat_reshaped = feat.view(B, C, -1)
        mean = feat_reshaped.mean(dim=2).view(B, C, 1, 1)
        std = feat_reshaped.std(dim=2).view(B, C, 1, 1) + self.eps
        return mean, std

    def content_loss(self, gen_feat, t):
        return nn.functional.mse_loss(gen_feat, t)

    def style_loss(self, gen_feat, style_feat):
        loss = 0.0
        for layer in self.style_layers:
            g_mean, g_std = self.calc_mean_std(gen_feat[layer])
            s_mean, s_std = self.calc_mean_std(style_feat[layer])
            loss += nn.functional.mse_loss(g_mean, s_mean) + nn.functional.mse_loss(g_std, s_std)
        return loss

    def forward(self, generated, t, style):
        gen_feat = self.encoder(generated)
        style_feat = self.encoder(style)
        # resize t cho khớp với gen_feat
        t_resized = nn.functional.interpolate(t, size=gen_feat['relu4_1'].shape[2:], mode='nearest')
        c_loss = self.content_loss(gen_feat['relu4_1'], t_resized)
        s_loss = self.style_loss(gen_feat, style_feat)

        total = self.alpha * c_loss + self.beta * s_loss
        return total, c_loss, s_loss

## Training

In [3]:
import os
import torch
from tqdm import tqdm
from torchvision.utils import save_image

def train_model(
    train_loader, test_loader,
    model, criterion, optimizer,
    device, num_epochs,
    save_dir="./checkpoints"
):
    os.makedirs(save_dir, exist_ok=True)

    # ====================== LOAD CHECKPOINT ======================
    checkpoint_path = os.path.join(save_dir, "latest_checkpoint.pth")
    start_epoch = 0
    best_val_loss = float("inf")

    if os.path.exists(checkpoint_path):
        print(f"🔄 Loading checkpoint from {checkpoint_path}")
        ckpt = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(ckpt["model"])
        optimizer.load_state_dict(ckpt["optimizer"])
        start_epoch = ckpt["epoch"] + 1
        best_val_loss = ckpt["best_val_loss"]
        print(f"➡ Continue from epoch {start_epoch}, best_val_loss={best_val_loss:.4f}")

    # ====================== TRAIN LOOP ======================
    for epoch in range(start_epoch, num_epochs):
        model.train()
        running_loss = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch} Training")

        for content, style in pbar:
            content, style = content.to(device), style.to(device)

            optimizer.zero_grad()
            generated, t = model(content, style)
            loss, c_loss, s_loss = criterion(generated, t, style)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "Content": f"{c_loss.item():.4f}",
                "Style": f"{s_loss.item():.4f}",
            })

        avg_train_loss = running_loss / len(train_loader)

        # ====================== VALIDATION ======================
        model.eval()
        val_loss = 0.0
        sample_saved = False

        with torch.no_grad():
            for batch_idx, (content, style) in enumerate(test_loader):
                content, style = content.to(device), style.to(device)
                generated, target_feat = model(content, style)
                loss, _, _ = criterion(generated, target_feat, style)
                val_loss += loss.item()

                if not sample_saved:
                    img_path = os.path.join(save_dir, f"epoch_{epoch}_sample.png")
                    save_image(generated.clamp(0,1), img_path)
                    sample_saved = True

        avg_val_loss = val_loss / len(test_loader)
        print(f"✅ Epoch {epoch} | Train: {avg_train_loss:.4f} | Val: {avg_val_loss:.4f}")

        # ====================== SAVE BEST MODEL ======================
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_path = os.path.join(save_dir, "best_model.pth")
            torch.save(model.state_dict(), best_path)
            print(f"🏆 Best model updated! Saved: {best_path}")

        # ====================== SAVE CHECKPOINT (Resume) ======================
        torch.save({
            "epoch": epoch,
            "model": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "best_val_loss": best_val_loss,
        }, checkpoint_path)

    print("🎯 Training Completed!")
    return best_val_loss

In [ ]:
import torchvision.transforms.functional as F
loader = get_dataloaders(content_folder="../data/coco2017",
                         style_folder="../data/wikiart_sampled", num_workers=32, batch_size=16, persistent_workers=False)
path_vgg_weights = "../models/vgg16-encoder.pth"
device = ("cuda:7" if torch.cuda.is_available() else "cpu")
criterion = AdaINLossMultiLayer(encoder=VGGEncoderMultiLayer(path_vgg_weights=path_vgg_weights, device=device), alpha=1.0, beta=10)
model = AdaINet(out_channels=3, path_vgg_weights=path_vgg_weights, device=device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4*0.5)
best_val_loss = train_model(
    train_loader=loader["train"],
    test_loader=loader["valid"],
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    num_epochs=100,
    save_dir="checkpoints_adaln"
)


## Testing

In [13]:
import os
from torchvision.utils import save_image
from tqdm import tqdm
import cv2
import numpy as np
import os, glob, random
import torch
from tqdm import tqdm
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F

def load_image(path, device):
    """Load 1 ảnh, resize và chuyển thành tensor [1,3,H,W]"""
    img = Image.open(path).convert("RGB")

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
    ])
    img = transform(img).unsqueeze(0).to(device)
    return img

def smooth_image(tensor):
    """tensor: [3,H,W] trong range [0,1]"""
    img = tensor.detach().cpu().permute(1,2,0).numpy()
    img = (img * 255).astype(np.uint8)
    img = cv2.bilateralFilter(img, d=7, sigmaColor=50, sigmaSpace=50)
    img = torch.tensor(img.astype(np.float32)/255).permute(2,0,1)
    return img

def unnormalize(tensor):
    mean = torch.tensor([0.485, 0.456, 0.406], device=tensor.device).view(1, 3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225], device=tensor.device).view(1, 3, 1, 1)
    return tensor * std + mean

def test_model(model, content_dir, style_dir, device, checkpoint_path, save_dir="./test_results"):
    os.makedirs(save_dir, exist_ok=True)
    content_paths = sorted(glob.glob(os.path.join(content_dir, "*.jpg")))
    style_paths = sorted(glob.glob(os.path.join(style_dir, "*.jpg")))
    content_paths = content_paths[:20]

    # Load checkpoint
    print(f"🔄 Loading model weights from {checkpoint_path}")
    ckpt = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(ckpt)
    model.to(device)
    model.eval()

    with torch.no_grad():
        for ci, content_path in enumerate(tqdm(content_paths, desc="Testing content")):
            # 👉 chọn ngẫu nhiên 1 style cho mỗi content
            style_path = random.choice(style_paths)

            # Load ảnh
            content = load_image(content_path, device)
            style = load_image(style_path, device)

            # Save kết quả
            pair_dir = os.path.join(save_dir, f"pair_{ci + 1}")
            os.makedirs(pair_dir, exist_ok=True)

            save_image(unnormalize(content).clamp(0, 1),
                       os.path.join(pair_dir, "content.jpg"))
            save_image(unnormalize(style).clamp(0, 1),
                       os.path.join(pair_dir, "style.jpg"))
            for alpha in [0.8, 1.0, 1.2]:
                generated, _ = model(content, style, alpha=alpha)

                # Resize output về đúng kích thước content
                _, _, h, w = content.shape
                resized = F.interpolate(generated, size=(h, w), mode='bilinear', align_corners=False)
                out = unnormalize(resized)
                # Save
                out_path = os.path.join(pair_dir, f"result_alpha_{alpha:.1f}.jpg")
                save_image(out.clamp(0,1), out_path)

    print(f"✅ Testing completed! Results saved in {save_dir}")


In [14]:
path_vgg_weights = "../models/vgg16-encoder.pth"
device = ("cuda:2" if torch.cuda.is_available() else "cpu")
model = AdaINet(out_channels=3, path_vgg_weights=path_vgg_weights, device=device).to(device)
test_model(
    model=model,
    content_dir="../data/coco2017/test",
    style_dir="../data/wikiart_sampled/test",
    device=device,
    checkpoint_path="./checkpoints_adaln/best_model.pth",
    save_dir="../results/adain"
)

🔄 Loading model weights from ./checkpoints_adaln/best_model.pth


Testing content: 100%|██████████| 20/20 [00:12<00:00,  1.62it/s]

✅ Testing completed! Results saved in ../results/adain
